# Analysis with Random Forest class 1st shot
### Thorowing in all potentials: including the kitchen sink (needs clean up)

In [1]:
import pandas as pd
import numpy as np
import time
import pydotplus
import plotly.graph_objs as go
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
from tqdm import tqdm
from pathlib import Path
from load_data import load_parse_save
from hlper_func import train_validate_test_split
from sklearn.model_selection import train_test_split
from scipy import stats
import pydotplus
import graphviz
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
from sklearn.metrics import roc_auc_score,mean_squared_error, mean_absolute_error
from sklearn.preprocessing import LabelBinarizer

### Load data & mark any prior & fail metrics as Failed set 

In [2]:
# Load Data from csv file & process via load_parse_save
tqdm.pandas(desc='load & parse csv')
#prev_df = load_parse_save(save_file='./parsed_dataframe.pkl', debug=True)
#prev_df.to_csv('/tmp/prev_maint_parsed.csv')

# Alternative is Load from Pickled set: Saves Time 
prev_df = pd.read_pickle('./parsed_dataframe.pkl')
#test_g_df = test_df[((test_df.daysDelta.isna()) | (test_df.daysDelta > 0.0))] #.to_csv('/tmp/test_base.csv')

### Add column Fail_set & mark it
This is a bit too close for comfort. 
Ideally, we would cut off data at least 7 days prior to Failure if possible, 
as one does NOT want to wait until last min. 
#### So, this will be an improvement for later
Only the data prev to fail is one group & other group is no Failure + post Failure
#### N.B.: This only works because there is only ONE failure for each device.
Must deal with multiple Fails on a device with a bit more sophistication

In [3]:
prev_df['Fail_set'] = prev_df.daysDelta.apply(lambda x: True if x <= 0.0 else False)

# look at devices level & create group level Stat!
### Need to think of better stats The idea is that extreme values on the metrics are likely indicator of an issue

In [46]:
# devices = pd.DataFrame(prev_df.device.value_counts().reset_index())
    
f =  lambda x: pd.Series({
      'Fails' : True if x['failure'].sum() > 0 else False,
      'count' : x['date'].count(),
      'm1_max': x['metric1'].max(), 'm1_mean': x['metric1'].mean(), 'm1_min': x['metric1'].min(), 'm1_std' : x['metric1'].std(),
      'm2_max': x['metric2'].max(), 'm2_mean': x['metric2'].mean(), 'm2_min': x['metric2'].min(), 'm2_std' : x['metric2'].std(),
      'm3_max': x['metric3'].max(), 'm3_mean': x['metric3'].mean(), 'm3_min': x['metric3'].min(), 'm3_std' : x['metric3'].std(),
      'm4_max': x['metric4'].max(), 'm4_mean': x['metric4'].mean(), 'm4_min': x['metric4'].min(), 'm4_std' : x['metric4'].std(),
      'm5_max': x['metric5'].max(), 'm5_mean': x['metric5'].mean(), 'm5_min': x['metric5'].min(), 'm5_std' : x['metric5'].std(),
      'm6_max': x['metric6'].max(), 'm6_mean': x['metric6'].mean(), 'm6_min': x['metric6'].min(), 'm6_std' : x['metric6'].std(),
      'm7_max': x['metric7'].max(), 'm7_mean': x['metric7'].mean(), 'm7_min': x['metric7'].min(), 'm7_std' : x['metric7'].std(),
      'm8_max': x['metric8'].max(), 'm8_mean': x['metric8'].mean(), 'm8_min': x['metric8'].min(), 'm8_std' : x['metric8'].std(),
      'm9_max': x['metric9'].max(), 'm9_mean': x['metric9'].mean(), 'm9_min': x['metric9'].min(), 'm9_std' : x['metric9'].std(),
  })
   
ggrp = prev_df.groupby(['device'])
groupd_prev_stat = ggrp.apply(f)
#groupd_prev_stat.head()

### Split the data into Train & Test, then use Random Forest to Predict
#### Not sure RF is a good model as results are volatile

In [47]:
y = groupd_prev_stat['Fails']
X = groupd_prev_stat[['count', 'm1_max' ,'m1_mean' ,'m1_min' ,'m2_max', 'm2_mean', 'm2_min', 
                      'm3_max', 'm3_mean', 'm3_min', 'm4_max', 'm4_mean', 'm4_min', 
                      'm5_max', 'm5_mean', 'm5_min', 'm6_max', 'm6_mean', 'm6_min', 
                      'm7_max', 'm7_mean', 'm7_min', 'm8_max', 'm8_mean', 'm8_min', 
                      'm9_max', 'm9_mean', 'm9_min']]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3) # 70% training and 30% test

                     
print(X_train.shape, X_test.shape)
# Model Accuracy, how often is the classifier correct?

clf=RandomForestClassifier(n_estimators=100)

#Train the model using the training sets y_pred=clf.predict(X_test)
clf.fit(X_train,y_train)

y_pred=clf.predict(X_test)

(818, 28) (351, 28)


## Accuracy of the model
#### N.B. General Accuracy is extremely misleading as imbalance class issue comes up
Only 20% ~ 50% of True fails & F1_score 25% ~ 55% achieved
Needs higher % on the True fails & F1 + Results needs to be less ***Volatile***
#### This is business decision as some business cannot tolerate any fails/downtime like electronic manufacturing.
In this case nearly or all potential fails must be investigated even if few dozen UnFails creep in for every REAL Failures.

In [48]:
from sklearn.metrics import f1_score

print("Accuracy: {:,.3f}%".format(100 * metrics.accuracy_score(y_test, y_pred)))

got_fail_df = pd.DataFrame(y_test)
got_fail_df['pred'] = y_pred
True_fails = got_fail_df.Fails.sum()
fail_correct_count = (got_fail_df[(got_fail_df.Fails == True)].sum(axis=1) > 1).sum()
print("Out of {} Fails, predicted {} or {:,.1f}% correctly".format(True_fails, fail_correct_count, 100*fail_correct_count/True_fails)) 

print("F1_score = {:,.1f}%".format(100 * f1_score(y_test, y_pred, 'binary')))

Accuracy: 92.593%
Out of 34 Fails, predicted 13 or 38.2% correctly
F1_score = 50.0%


# Boosted Models
For imbalanced data usual techniques are:
- Random over & under Resampling (Either can disregard useful data w/ undersample or overfit w/ oversample)
- Clustered over-sampling (outperfoms underampling & outperforms it, but often over fits)
- SMOT (Synthetic Minority Over-sampling Technique): Really difficult wiht hi-dim data
- Bagging: Only works if based classifiers distribution is not extreme
- Various Boosting (Ada, Gradient, & XG): Ada Sensitive, XG fast & less prone to issues

In [49]:
from xgboost import XGBClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3) # 70% training and 30% test

# Started out w/ AdaBoost & not saftifying
#classifier = AdaBoostClassifier(
#    DecisionTreeClassifier(max_depth=1),
#    n_estimators=200
#)

classifier = XGBClassifier()

classifier.fit(X_train, y_train)
predictions = classifier.predict(X_test)

### *** Volatile F1 65~75% ***
- The results are better F1 score & hit ratio, but still very data random selection dependent.
- Likely results of using max/min from the metrics + needs better agg_metrics/factors
- The confusion matrix seems to indicate that model is still dominated by dominant classifier.
- Indicating that results may be better with Either sampling or SMOT 
- Indicating that results may be better with Either sampling or SMOT 

In [44]:
print(confusion_matrix(y_test, predictions))
print("Accuracy: {:,.3f}%".format(100 * metrics.accuracy_score(y_test, predictions)))

print("Training data had {} data points & {} fails".format(y_train.shape[0], y_train.sum()))
print("Testing  data had {} data points & {} fails".format(y_test.shape[0], y_test.sum()))
got_fail_df = pd.DataFrame(y_test)
got_fail_df['pred'] = predictions
True_fails = got_fail_df.Fails.sum()
fail_correct_count = (got_fail_df[(got_fail_df.Fails == True)].sum(axis=1) > 1).sum()
print("Out of {} Fails, predicted {} or {:,.1f}% correctly".format(True_fails, fail_correct_count, 100*fail_correct_count/True_fails)) 

print("F1_score = {:,.1f}%".format(100 * f1_score(y_test, predictions, 'binary')))

[[315   5]
 [ 11  20]]
Accuracy: 95.442%
Training data had 818 data points & 75 fails
Testing  data had 351 data points & 31 fails
Out of 31 Fails, predicted 20 or 64.5% correctly
F1_score = 71.4%
